In [12]:
import pyscf
import slowquant.SlowQuant as sq
from qiskit_nature.second_q.mappers import JordanWignerMapper
from slowquant.qiskit_interface.interface import QuantumInterface
from slowquant.qiskit_interface.wavefunction import WaveFunction
#from slowquant.qiskit_interface.linear_response.projected import quantumLR

from qiskit_ibm_runtime.fake_provider import FakeTorino

from qiskit.primitives import Estimator

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.primitives import Sampler
from qiskit_aer.primitives import SamplerV2

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import random

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

In [2]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 0.735 0.0 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 1}, ISA = True)

#Remember to change the wavefunction to the correct molecule

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (2, 2),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
)

WF.run_vqe_2step("RotoSolve")

#no noise
nonclif_ground_state_energy = WF.energy_elec
print("Non-Clifford Ground state energy:", nonclif_ground_state_energy)

#device noise
# Update the primitive with simulated noise
backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)    
# Calculate the ground state energy using the noisy simulator
noisy_nonclif_ground_state_energy = WF.energy_elec
print("Noisy Non-Clifford Ground state energy:",noisy_nonclif_ground_state_energy)

converged SCF energy = -1.116998996754
Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3
Full optimization
Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------Ansatz optimization
--------Iteration # | Iteration time [s] | Electronic energy [Hartree]


/tmp/ipykernel_589882/317835389.py:5: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


--------     1      |         3.92       |     -1.8572750302023797    
Non-Clifford Ground state energy: -1.8572750302023793
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [103, 93, 102, 104]
 Non-local gates:     36
 Transpilation strategy: External PassManager
 Primitive backend:   AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['id', 'measure', 'sx', 'x', 'cz', 'reset']>)

Noisy Non-Clifford Ground state energy: -1.732580263299815


In [5]:
#prepare training set
clif_ground_state_energies = []
noisy_clif_ground_state_energies = []
n_replacements = 1

for n in range(40):
    QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 1}, ISA = True)

    WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (2, 2),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    )
    indices_to_replace = random.sample(range(len(WF.ansatz_parameters)), n_replacements)
    for index in indices_to_replace:
        WF.ansatz_parameters[index] = random.uniform(0, 2*np.pi)
    print(WF.ansatz_parameters)
    clif_ground_state_energy = WF.energy_elec
    clif_ground_state_energies.append(clif_ground_state_energy)
    
    backend = FakeTorino()
    QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
    QI.do_postselection = False
    QI.do_M_mitigation = False
    noise_model = NoiseModel.from_backend(backend)
    sampler = Sampler(backend_options={"noise_model":noise_model})
    WF.change_primitive(sampler)   
    noisy_clif_ground_state_energy = WF.energy_elec
    noisy_clif_ground_state_energies.append(noisy_clif_ground_state_energy)

Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3
[0.0, 6.008221010628369, 0.0]
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [103, 93, 102, 104]
 Non-local gates:     36
 Transpilation strategy: External PassManager
 Primitive backend:   AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['id', 'measure', 'sx', 'x', 'cz', 'reset']>)

Number of shots is None. Ideal sim

In [6]:
#data preparation
nonclif_ground_state_energies = [nonclif_ground_state_energy for _ in range(len(clif_ground_state_energies))]
noisy_nonclif_ground_state_energies = [noisy_nonclif_ground_state_energy for _ in range(len(noisy_clif_ground_state_energies))]
X = np.array(noisy_clif_ground_state_energies).reshape(-1, 1)
y = np.array(clif_ground_state_energies)
X_test = np.array(noisy_nonclif_ground_state_energies).reshape(-1, 1)
y_test = np.array(nonclif_ground_state_energies)

In [7]:
#square regression with a linear ansatz
def cost_function(a, X_exact, X_noisy):
    a1, a2 = a  # 将参数 a 拆分为 a1 和 a2
    # 计算代价函数 C 的值
    C = np.sum((X_exact - (a1 * X_noisy + a2))**2)
    return C
X_exact = np.array(clif_ground_state_energies)
X_noisy = np.array(noisy_clif_ground_state_energies)
# 初始猜测值 a1 和 a2
initial_guess = [1, 0]
# 最小化代价函数
result = minimize(cost_function, initial_guess, args=(X_exact, X_noisy))
# 得到最优的 a1 和 a2
a1_optimal, a2_optimal = result.x
print(f"最优的 a1 值: {a1_optimal}")
print(f"最优的 a2 值: {a2_optimal}")
y_pred_reg = a1_optimal*noisy_nonclif_ground_state_energy + a2_optimal
print('result of square regression:', y_pred_reg)

最优的 a1 值: 1.1181651001575332
最优的 a2 值: 0.08791165758148937
result of square regression: -1.8493991260621137


In [11]:
# Weighted Least Squares
def weighted_cost_function(a, X_exact, X_noisy, weights):
    a1, a2 = a  # 拆分参数
    residuals = X_exact - (a1 * X_noisy + a2)
    # 加权平方和
    C = np.sum(weights * residuals**2)
    return C

# 输入数据
X_exact = np.array(clif_ground_state_energies)
X_noisy = np.array(noisy_clif_ground_state_energies)

# 假设权重为噪声的不确定性 (例如误差) 的倒数平方
# 示例权重：可以根据具体问题替换为真实的不确定性
noise_uncertainties = np.random.rand(len(X_noisy)) * 0.1  # 示例噪声
weights = 1 / (noise_uncertainties**2)

# 初始猜测值
initial_guess = [1, 0]

# 使用 scipy.optimize.minimize 进行优化
result = minimize(weighted_cost_function, initial_guess, args=(X_exact, X_noisy, weights))

# 提取最优参数
a1_optimal, a2_optimal = result.x

# 打印结果
print(f"最优的 a1 值: {a1_optimal}")
print(f"最优的 a2 值: {a2_optimal}")

# 对新的数据进行预测
y_pred_reg = a1_optimal * noisy_nonclif_ground_state_energy + a2_optimal
print('加权最小二乘回归结果:', y_pred_reg)

最优的 a1 值: 1.1300079361302093
最优的 a2 值: 0.0964922574662909
加权最小二乘回归结果: -1.8613371900450677


In [13]:
#Nonlinear Fitting
# 定义非线性拟合函数
def nonlinear_model(X_noisy, a1, a2, a3):
    return a1 * X_noisy**2 + a2 * X_noisy + a3

# 输入数据
X_exact = np.array(clif_ground_state_energies)
X_noisy = np.array(noisy_clif_ground_state_energies)

# 使用 curve_fit 进行非线性拟合
# 初始猜测值 [a1, a2, a3]
initial_guess = [1, 1, 0]
params, covariance = curve_fit(nonlinear_model, X_noisy, X_exact, p0=initial_guess)

# 提取拟合参数
a1_optimal, a2_optimal, a3_optimal = params
print(f"最优的 a1 值: {a1_optimal}")
print(f"最优的 a2 值: {a2_optimal}")
print(f"最优的 a3 值: {a3_optimal}")

# 对新的数据进行预测
y_pred_reg = nonlinear_model(noisy_nonclif_ground_state_energy, a1_optimal, a2_optimal, a3_optimal)
print('非线性拟合结果:', y_pred_reg)

最优的 a1 值: -0.016148543042210584
最优的 a2 值: 1.0850204215420318
最优的 a3 值: 0.07518486345584346
非线性拟合结果: -1.8531753556948938


In [9]:
#XGBoost
xgb_model = XGBRegressor(max_depth=20, eta=0.0005, subsample=0.7, colsample_bytree=0.7, objective='reg:absoluteerror', n_estimators=1000, random_state=42)
xgb_model.fit(X, y)
y_pred_xgb = xgb_model.predict(X_test)
# 计算均方误差
#mse_corrected_XGB = mean_squared_error(y_test, y_pred_xgb)
#print('mse:', mse_corrected_XGB)
print('result of xgb:', y_pred_xgb[0])

result of xgb: -1.3650149


In [10]:
#MLP
# 归一化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# 训练MLP模型
mlp_model = MLPRegressor(hidden_layer_sizes=(120, 60), activation='relu', solver='adam', max_iter=2000, alpha=0.001, random_state=42)
mlp_model.fit(X_scaled, y)
X_test_scaled = scaler.transform(X_test)
y_pred_mlp = mlp_model.predict(X_test_scaled)
#mse_corrected_MLP = mean_squared_error(y_test, y_pred_mlp)
#print('mse:', mse_corrected_MLP)
print('result of mlp:', y_pred_mlp[0])

result of mlp: -1.9375060172476797
